# Forecasting tax avoidance rates by international listed companies

## Problem description

**Tax avoidance is not a crime!** Companies that carry out this act are on the verge of law. By definition: tax avoidance consists in carrying out economic activities in such a way that they are understood in a different way from tax regulations, in order to reduce the tax burden. International tax avoidance arose as a result of globalization and liberalization of economic systems of countries, weakening trade barriers and development of new technologies. Tax avoidance is achieved through aggressive **tax optimization** (e.g. tax havens, double taxation agreements, etc.). 

In this study, we will check whether we can **forecast for one year ahead the level of tax avoidance by a group of companies** listed on stock exchanges using shallow Machine Learning models. As the econometric research shows, this problem is non-trivial, and the most important determinants result from the financial statements per se. The question arises whether any additional data sources can be used to forecast this problem. Forecasting such a phenomenon may be particularly important for the tax authorities and legislators. It helps to create rules that would fight against tax avoidance!

There are many ways in the literature to measure tax avoidance. All of them have their advantages and disadvantages. However, the most popular metric appears to be Effective Tax Rate (ETR) = $\dfrac{\textrm{total tax expenses}}{\textrm{pre-tax income}} $. Due to this formula, ETR has values in the range [0,1]. This measure applies directly to each jurisdiction and it is based on annual data published in the financial statements and this involves an annual change in the effective tax rate, or failure to determine it , in the case of negative income tax resulting from current tax overruns of deferred tax assets. ETR will be used as the target/endogenous variable in this study. Therefore, the following evaluation metrics for the given problem were selected: Mean Absolute Error (MAE), Root Mean Square error (RMSE). The choice was not accidental: MAE allows for relatively easy interpretation, while RMSE punishes model for large individual errors, which in the case of ETR forecasting may be crucial. For this case, absolute measures seem to be a more correct approach than relative ones. Nevertheless, the most important metric will be RMSE.

This problem is a classic panel problem (many companies and many years).

## Dataset description

The database used in the study was created for the purposes of the paper "Determinants of multinational tax avoidance" (Agnieszka Teterycz, PhD. Anna Białek) on the basis of data retrieved from the Bloomberg database, OECD and PWC reports. Information on the introduction of regulations concerning foreign controlled companies (CFCs) in the analysed countries has been selected from OECD reports. From the reports prepared by PWC, data specifying the number of double taxation agreements signed in the analysed countries were selected. The dataset gather companies included in WIG, DAX, UK100, CAC40 and ATX indices listed on stock exchanges in Poland, Germany, Great Britain, France and Austria in 2005-2017. All companies from the financial (including banks) and insurance sectors were excluded from the analysis, as well as those with missing data in the explanatory variables. In addition, the observations that took a negative value for the financial result before tax and income tax were removed in order to avoid situations where a negative value of ETR would be difficult to interpret. The above exclusions and removal of outlier observations reduced the sample for panel data from 7 800 to 4 719 observations. Panel data are balanced (13 years x 363 companies). Inputation process was applied - using medians and means at the company level. Authors also used fill forward interpolation at the beginning of the time series.

#### Columns description 

* index - technical index
* ticker - company ticker from stock exchange
* Nazwa2 - full name of a company
* sektor - business sector of a company
* rok - year
* gielda - the stock exchange from which the company originates {1: Warsaw, 2: London, 3: Frankfurt, 4: Paris, 5: Vienna}
* ta - total assets of a company 
* txt - total tax expenses of a company 
* pi - pre-tax income of a company 
* str - statutory tax rate of a company 
* xrd - research and development expenditure of a company 
* ni - net income of a company 
* ppent - property plant and equipment net of a company  
* intant - total intangible assets of a company 
* dlc - long term debt of a company 
* dltt - short term debt of a company 
* capex - capital expenditures of a company 
* revenue - revenue of a company 
* cce - cash and cash equivalents of a company 
* adv - advertising expenses of a company  
* etr - effective tax rate of a company
* diff - statutory tax rate - effective tax rate
* roa - return of assets of a company
* lev - leverage of a company
* intan - intangible assets/total assets
* rd - research and development expenditure/total assets
* ppe - property plant and equipment/total assets
* sale - log(revenue of a company/total assets)
* cash_holdings - cash and cash equivalents of a company/total assets
* adv_expenditures - advertising expenses/total assets
* capex2 - capex/property plant and equipment
* cfc - control foreign companies by a company
* dta - double taxation agreements of a company 
* capex2_scaled - scaled capex2

the rest of the columns are technical and redundant, so they will be deleted!

## Dependencies loading

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)

## Data preparation

### Data loading

In [35]:
df = pd.read_stata("../data/tax_avoidance.dta")

In [36]:
df.sample(10)

,index,Ticker,Nazwa2,sektor,rok,gielda,ta,txt,pi,str,xrd,ni,ppent,intant,dlc,dltt,capex,revenue,cce,adv,etr,diff,roa,lev,intan,rd,ppe,sale,cash_holdings,adv_expenditure,capex2,cfc,dta,capex2_scaled,firm_id,firma_id,rok2005,rok2006,rok2007,rok2008,rok2009,rok2010,rok2011,rok2012,rok2013,rok2014,rok2015,rok2016,rok2017,industry,industry1,capex1,roa1,country1,country2,country3,country4,country5,industry11,industry12,industry13,industry14,industry15,industry16,industry17,industry18,industry19,industry20,diff1,diff2,diff3,_est_random,_est_fixed
4702,3604,EZJ LN Equity,easyJet PLC,consumer discretionary,2014,3,4482.000000,131.000,581.000000,0.2200,450.000000,2542.000000,0.000000,478.000,472.000000,91.000000,426.000,4527.000000,424.000000,0.0,0.225473,-0.005473,0.567158,0.125614,0.106649,0.100402,0.000000,0.698155,0.094601,0.0,0.000000,1,1,0.000000,easyJet PLC,easyJet PLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,consumer discretionary,consumer discretionary,6.056784,0.567158,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.005473,-0.005473,-0.005473,1,1
2547,1417,MDI PW Equity,MDI Energia SA,utilities,2017,1,105.995003,1.276,5.796000,0.1900,0.000000,4.520000,0.309000,0.000,1.692000,26.990000,0.151,174.210007,7.100000,0.0,0.220152,-0.030152,0.042644,0.270598,0.000000,0.000000,0.002915,0.972130,0.066984,0.0,0.488673,1,0,0.000454,MDI Energia SA,MDI Energia SA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,utilities,utilities,0.140631,0.042644,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.030152,-0.030152,-0.030152,1,1
4180,4074,TW/ LN Equity,Taylor Wimpey PLC,consumer discretionary,2012,3,3550.300049,-24.400,204.199997,0.2450,228.600006,7.100000,0.000000,5.200,249.399994,0.000000,3.500,2019.000000,190.399994,0.0,0.000000,0.364491,0.002000,0.070248,0.001465,0.064389,0.000000,0.450237,0.053629,0.0,0.000000,1,1,0.000000,Taylor Wimpey PLC,Taylor Wimpey PLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,consumer discretionary,consumer discretionary,1.504077,0.002000,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.364491,0.364491,0.364491,1,1
473,2646,BAS GY Equity,BASF SE,materials,2010,2,59393.000000,2299.000,7373.000000,0.2941,1492.000000,4557.000000,17241.000000,12245.000,11719.000000,3379.000000,2548.000,63873.000000,1493.000000,0.0,0.311813,-0.017713,0.076726,0.254205,0.206169,0.025121,0.290287,0.730168,0.025138,0.0,0.147787,1,0,0.000137,BASF SE,BASF SE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,materials,materials,7.843456,0.076726,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.017713,-0.017713,-0.017713,1,1
3605,3193,RHM GY Equity,Rheinmetall AG,consumer discretionary,2009,2,3835.000000,6.000,-46.000000,0.2944,198.000000,-58.000000,1044.000000,555.000,180.000000,353.000000,145.000,3420.000000,557.000000,0.0,0.000000,0.424835,-0.015124,0.138983,0.144720,0.051630,0.272229,0.637521,0.145241,0.0,0.138889,1,0,0.000129,Rheinmetall AG,Rheinmetall AG,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,consumer discretionary,consumer discretionary,4.983607,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.424835,0.424835,0.424835,1,1
2470,1312,LEN PW Equity,Lena Lighting SA,industrials,2005,1,64.063004,2.626,37.828999,0.1900,0.000000,35.202999,18.732000,0.637,0.000000,0.000000,8.931,111.789001,4.860000,0.0,0.069418,0.120582,0.549506,0.000000,0.009943,0.000000,0.292400,1.009776,0.075863,0.0,0.476778,0,0,0.000442,Lena Lighting SA,Lena Lighting SA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,industrials,industrials,2.295661,0.549506,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.120582,0.120582,0.120582,1,1
1348,796,ENT PW Equity,Enter Air SA,industrials,2014,1,223.029999,4.608,22.024000,0.1900,0.000000,17.431000,91.828003,0.000,34.891998,29.122999,3.562,731.497009,32.285000,0.0,0.209226,-0.019226,0.078155,0.287024,0.000000,0.000000,0.411729,1.453910,0.144756,0.0,0.038790,0,0,0.000036,Enter Air SA,Enter Air SA,0.0,

### Dataset adjustment

##### Removing redundant variables

In [37]:
df.columns

Index(['index', 'Ticker', 'Nazwa2', 'sektor', 'rok', 'gielda', 'ta', 'txt',
       'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex',
       'revenue', 'cce', 'adv', 'etr', 'diff', 'roa', 'lev', 'intan', 'rd',
       'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc',
       'dta', 'capex2_scaled', 'firm_id', 'firma_id', 'rok2005', 'rok2006',
       'rok2007', 'rok2008', 'rok2009', 'rok2010', 'rok2011', 'rok2012',
       'rok2013', 'rok2014', 'rok2015', 'rok2016', 'rok2017', 'industry',
       'industry1', 'capex1', 'roa1', 'country1', 'country2', 'country3',
       'country4', 'country5', 'industry11', 'industry12', 'industry13',
       'industry14', 'industry15', 'industry16', 'industry17', 'industry18',
       'industry19', 'industry20', 'diff1', 'diff2', 'diff3', '_est_random',
       '_est_fixed'],
      dtype='object')

In [38]:
df.drop(columns=['index', 'firm_id', 'firma_id', 'rok2005', 'rok2006',
       'rok2007', 'rok2008', 'rok2009', 'rok2010', 'rok2011', 'rok2012',
       'rok2013', 'rok2014', 'rok2015', 'rok2016', 'rok2017', 'industry',
       'industry1', 'capex1', 'roa1', 'country1', 'country2', 'country3',
       'country4', 'country5', 'industry11', 'industry12', 'industry13',
       'industry14', 'industry15', 'industry16', 'industry17', 'industry18',
       'industry19', 'industry20', 'diff1', 'diff2', 'diff3', '_est_random',
       '_est_fixed'], inplace = True)

df.shape

(4719, 33)

##### Checking if every variable has proper type

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4719 entries, 0 to 4718
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Ticker           4719 non-null   object 
 1   Nazwa2           4719 non-null   object 
 2   sektor           4719 non-null   object 
 3   rok              4719 non-null   int16  
 4   gielda           4719 non-null   int8   
 5   ta               4719 non-null   float64
 6   txt              4719 non-null   float64
 7   pi               4719 non-null   float64
 8   str              4719 non-null   float64
 9   xrd              4719 non-null   float64
 10  ni               4719 non-null   float64
 11  ppent            4719 non-null   float64
 12  intant           4719 non-null   float64
 13  dlc              4719 non-null   float64
 14  dltt             4719 non-null   float64
 15  capex            4719 non-null   float64
 16  revenue          4719 non-null   float64
 17  cce           

#### Endogenous variable shifting

In [40]:
df.sort_values(by = ["Ticker","rok"],inplace = True)
df["etr"] = df["etr"].shift(-1)
df = df[df.rok != 2017]

df.shape

(4356, 33)

### External data adding

It was assumed in accordance with the effective market hypothesis that all information about specific companies is included in their book values at the end of the year, i.e. analyzing additional individual stock market data would rather not have the desired effect. In addition, company names (tickers) are not perfect and in some cases it is very difficult to find the right company on websites such as: stooq.pl. Therefore, the bulk approach is not optimal, and manual download would take too much time. Therefore, it was decided to choose variables that will testify to the condition of selected stock exchanges and the entire economy in each country.

**We checked that all added data is available no later than 3 months after the end of a given year. There is no date leakag here !!! The company's financial year should be taken into account, not the calendar year!**

#### First source: World Bank, V-Dem index, Polity index, BR index, BMR index
We assume that variables like: democracy indices (proxy for investment moods), GDP growth, GDP per capita and Inflation (proxy for business cycle ) might be significant in case of tax avoidance predictions. Data were gathered for mentioned above 5 countries (2005-2017). This data chunk is a part of our own dataset, which collects determinants of democracy. We don't want to share it right now, so we slice it.

In [41]:
df_extra0 = pd.read_csv("../data/external_dataset.csv")

In [42]:
df_extra0.head()

,country_name,year,y_v2x_polyarchy,y_e_p_polity,y_BR_Democracy,y_BMR_democracy,WB_GDPgrowth,WB_GDPpc,WB_Inflation
0,Austria,2005,0.855,10.0,1.0,1.0,2.244065,38403.133877,2.299139
1,Austria,2006,0.863,10.0,1.0,1.0,3.454042,40635.281816,1.441547
2,Austria,2007,0.885,10.0,1.0,1.0,3.727415,46855.771745,2.168556
3,Austria,2008,0.884,10.0,1.0,1.0,1.460424,51708.765754,3.215950
4,Austria,2009,0.894,10.0,1.0,1.0,-3.764578,47963.179402,0.506308


In [43]:
df_extra0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country_name     65 non-null     object 
 1   year             65 non-null     int64  
 2   y_v2x_polyarchy  65 non-null     float64
 3   y_e_p_polity     65 non-null     float64
 4   y_BR_Democracy   65 non-null     float64
 5   y_BMR_democracy  44 non-null     float64
 6   WB_GDPgrowth     65 non-null     float64
 7   WB_GDPpc         65 non-null     float64
 8   WB_Inflation     65 non-null     float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.7+ KB


In [44]:
df_extra0.country_name.unique()

array(['Austria', 'France', 'Germany', 'Poland', 'United Kingdom'],
      dtype=object)

In [45]:
df_extra0.country_name = df_extra0.country_name.map({"Poland":1,"United Kingdom":2,"Germany":3,"France":4,"Austria":5})

In [46]:
df_extra0.country_name.unique()

array([5, 4, 3, 1, 2], dtype=int64)

In [47]:
df.shape

(4356, 33)

In [48]:
df = pd.merge(df, df_extra0, left_on = ["gielda","rok"], right_on= ["country_name","year"], how = "left")
df.drop(columns = ["country_name","year"],inplace=True)

In [49]:
df.shape

(4356, 40)

#### Second source: main stock market indexes for Poland (WIG), Germany (DAX), Great Britain (FTSE), France (CAC) and Austria (ATX)
We assume that variables like yearly rates of returns might be additional proxy for market condition in each country.

In [50]:
cac = pd.read_csv("../data/external_dataset1/^cac_y.csv")
dax = pd.read_csv("../data/external_dataset1/^dax_y.csv")
a5c = pd.read_csv("../data/external_dataset1/a5_c_y.csv")
wig = pd.read_csv("../data/external_dataset1/wig_y.csv")
xf = pd.read_csv("../data/external_dataset1/x_f_y.csv")

In [51]:
cac["Date"] = pd.to_datetime(cac.Data)
cac["year"] = cac.Date.dt.year
cac["rr"] = (cac.Zamkniecie - cac.Otwarcie)/cac.Otwarcie
cac["country"] = 4

dax["Date"] = pd.to_datetime(dax.Data)
dax["year"] = dax.Date.dt.year
dax["rr"] = (dax.Zamkniecie - dax.Otwarcie)/dax.Otwarcie
dax["country"] = 3

a5c["Date"] = pd.to_datetime(a5c.Data)
a5c["year"] = a5c.Date.dt.year
a5c["rr"] = (a5c.Zamkniecie - a5c.Otwarcie)/a5c.Otwarcie
a5c["country"] = 5

wig["Date"] = pd.to_datetime(wig.Data)
wig["year"] = wig.Date.dt.year
wig["rr"] = (wig.Zamkniecie - wig.Otwarcie)/wig.Otwarcie
wig["country"] = 1

xf["Date"] = pd.to_datetime(xf.Data)
xf["year"] = xf.Date.dt.year
xf["rr"] = (xf.Zamkniecie - xf.Otwarcie)/xf.Otwarcie
xf["country"] = 2

In [52]:
tmp = pd.concat([cac,dax,a5c,wig,xf])
tmp = tmp[["year","rr","country"]]
tmp.rename(columns={"rr":"rr_per_country"},inplace=True)

In [53]:
df = pd.merge(df, tmp, left_on = ["rok","gielda"], right_on= ["year","country"], how = "left")
df.drop(columns = ["year","country"],inplace=True)

In [54]:
df.shape

(4356, 41)

#### Third source: S&P indexes for sectors
We believe that S&P indexes for sectors might be good proxies for each sector condition during each year!

In [55]:
df.sektor.value_counts()

consumer discretionary    924
industrials               816
materials                 672
technology                444
consumer staples          324
real estate               312
communication             300
health care               228
utilities                 204
energy                    132
Name: sektor, dtype: int64

In [56]:
def data_adjuster(tmp, sec):
    foo = tmp["May 20"].str.split(" ", n = 1, expand = True)
    tmp["0"] = foo[0]
    tmp["1"] = foo[1]
    tmp = tmp[tmp["0"].isin(["Dec","Jan"])]
    tmp["year"] = ("20"+tmp["1"]).astype(int)
    tmp = tmp[~tmp["year"].isin([2020,2019,2018,2017,2003])]
    tmp.iloc[:,2] = tmp.iloc[:,2].shift(-1)
    tmp["rr"] = (tmp.iloc[:,1] - tmp.iloc[:,2] )/ tmp.iloc[:,2]
    tmp = tmp[tmp["0"] == "Dec"] 
    tmp["sector"] = sec
    tmp = tmp[["sector","year","rr"]]
    return tmp

In [57]:
sp500cd = data_adjuster(pd.read_excel("../data/external_dataset2/S&P 500 Consumer Discretionary.xlsx", dtype = {"May 20":str}), "consumer discretionary")
sp500ind = data_adjuster(pd.read_excel("../data/external_dataset2/S&P 500 Industrial.xlsx", dtype = {"May 20":str}), "industrials")
sp500mat = data_adjuster(pd.read_excel("../data/external_dataset2/S&P 500 Materials.xlsx", dtype = {"May 20":str}), "materials")
sp500tech = data_adjuster(pd.read_excel("../data/external_dataset2/S&P 500 Information Technology.xlsx", dtype = {"May 20":str}), "technology")
sp500ics = data_adjuster(pd.read_excel("../data/external_dataset2/S&P 500 Consumer Discretionary.xlsx", dtype = {"May 20":str}), "consumer staples")
sp500re = data_adjuster(pd.read_excel("../data/external_dataset2/S&P 500 Real Estate.xlsx", dtype = {"May 20":str}), "real estate")
#for communication we will use S&P 500 because communication index is not avaliable for years 2005-2017
sp500comm = data_adjuster(pd.read_excel("../data/external_dataset2/S&P 500.xlsx", dtype = {"May 20":str}, thousands=","), "communication")
sp500hc = data_adjuster(pd.read_excel("../data/external_dataset2/S&P 500 Health Care.xlsx", dtype = {"May 20":str}), "health care")
sp500ut = data_adjuster(pd.read_excel("../data/external_dataset2/S&P 500 Utilities.xlsx", dtype = {"May 20":str}), "utilities")
sp500en = data_adjuster(pd.read_excel("../data/external_dataset2/S&P 500 Energy.xlsx", dtype = {"May 20":str}), "energy")

C:\Users\wozni\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [58]:
sp = pd.concat([sp500cd, sp500ind, sp500mat, sp500tech, sp500ics, sp500re, sp500comm, sp500hc, sp500ut, sp500en])
sp.rename(columns = {"rr":"rr_per_sector"},inplace=True)
sp = sp[sp["year"] != 2004]

In [59]:
df = pd.merge(df, sp, left_on = ["rok","sektor"], right_on= ["year","sector"], how = "left")
df.drop(columns = ["year","sector"],inplace=True)

In [60]:
df.shape

(4356, 42)

In [61]:
df.head()

,Ticker,Nazwa2,sektor,rok,gielda,ta,txt,pi,str,xrd,ni,ppent,intant,dlc,dltt,capex,revenue,cce,adv,etr,diff,roa,lev,intan,rd,ppe,sale,cash_holdings,adv_expenditure,capex2,cfc,dta,capex2_scaled,y_v2x_polyarchy,y_e_p_polity,y_BR_Democracy,y_BMR_democracy,WB_GDPgrowth,WB_GDPpc,WB_Inflation,rr_per_country,rr_per_sector
0,11B PW Equity,11 bit studios SA,communication,2005,1,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469,0.877,10.0,1.0,1.0,3.493668,8021.003655,2.183799,0.334868,0.0300102
1,11B PW Equity,11 bit studios SA,communication,2006,1,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469,0.884,10.0,1.0,1.0,6.179641,9038.730847,1.284694,0.421396,0.136194
2,11B PW Equity,11 bit studios SA,communication,2007,1,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469,0.887,10.0,1.0,1.0,7.034828,11255.436698,2.458743,0.104220,0.0354929
3,11B PW Equity,11 bit studios SA,communication,2008,1,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469,0.869,10.0,1.0,1.0,4.249609,14001.446882,4.164972,-0.511178,-0.384695
4,11B PW Equity,11 bit studios SA,communication,2009,1,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.188487,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469,0.878,10.0,1.0,1.0,2.820260,11527.593228,3.795392,0.429716,0.234897


#### Dataset splitting

We decided to split dataset into:
 * **train (& validation) dataset** - 2005 - 2015 (exogenous notation) x 364 companies
 * **test (out of sample / out of time) dataset**  - 2016 (exogenous notation) x 364 companies 

Test dataset will be used **only** for the final predictions! The authors assume that during the entire study they do not have access to it and do not study its statistical properties.

In [62]:
df_train = df[df.rok != 2016]
df_test = df[df.rok == 2016]

In [63]:
df_train.shape

(3993, 42)

In [64]:
df_test.shape

(363, 42)

## Initial descriptive analyses of the data

In [107]:
df.head()

,Ticker,Nazwa2,sektor,rok,gielda,ta,txt,pi,str,xrd,ni,ppent,intant,dlc,dltt,capex,revenue,cce,adv,etr,diff,roa,lev,intan,rd,ppe,sale,cash_holdings,adv_expenditure,capex2,cfc,dta,capex2_scaled
13,11B PW Equity,11 bit studios SA,communication,2005,1,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469
14,11B PW Equity,11 bit studios SA,communication,2006,1,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469
15,11B PW Equity,11 bit studios SA,communication,2007,1,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469
16,11B PW Equity,11 bit studios SA,communication,2008,1,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.196193,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469
17,11B PW Equity,11 bit studios SA,communication,2009,1,21.127613,1.24185,6.329725,0.19,0.0,5.0879,0.276275,4.1959,0.0,0.0,2.223413,11.873301,12.142975,0.0,0.188487,-0.006193,0.240818,0.0,0.198598,0.0,0.013076,0.445954,0.574744,0.0,8.047824,0,0,0.007469


In [109]:
df.sektor.value_counts()

consumer discretionary    924
industrials               816
materials                 672
technology                444
consumer staples          324
real estate               312
communication             300
health care               228
utilities                 204
energy                    132
Name: sektor, dtype: int64